In [1]:
# installs TensorFlow
!pip install tensorflow==2.15.0 --ignore-installed


Defaulting to user installation because normal site-packages is not writeable
  Obtaining dependency information for tensorflow==2.15.0 from https://files.pythonhosted.org/packages/93/c0/a774286d0383419f558deb27096e5de9f9facd6c27df8e9f9af6fba2f77e/tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata
  Using cached tensorflow-2.15.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.4 kB)
  Obtaining dependency information for absl-py>=1.0.0 from https://files.pythonhosted.org/packages/f6/d4/349f7f4bd5ea92dab34f5bb0fe31775ef6c311427a14d5a5b31ecb442341/absl_py-2.2.2-py3-none-any.whl.metadata
  Using cached absl_py-2.2.2-py3-none-any.whl.metadata (2.6 kB)
  Obtaining dependency information for astunparse>=1.6.0 from https://files.pythonhosted.org/packages/2b/03/13dde6512ad7b4557eb792fbcf0c653af6076b81e5941d36ec61f7ce6028/astunparse-1.6.3-py2.py3-none-any.whl.metadata
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)

In [2]:
#installs many libraries
!pip install tensorflow==2.15.0 numpy==1.26.4 matplotlib==3.8.2 scikit-learn==1.4.2 seaborn==0.13.2 pandas==2.2.1 scipy==1.12.0


Defaulting to user installation because normal site-packages is not writeable


In [3]:
#all key libraries you need to run the code
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import numpy as np
import os
import zipfile
import random
import shutil
from pathlib import Path

2025-04-22 01:06:35.060432: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-22 01:06:35.160474: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-04-22 01:06:35.160562: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-04-22 01:06:35.167934: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-04-22 01:06:35.196092: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2025-04-22 01:06:35.197377: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [4]:
# upload zip file first
# Define the zip file name and the directory to extract to
# uploading all the images 
zip_path = "american-sign-language-letters-subdir.zip"  # replace with your actual zip filename
extract_to = "data"  # or any folder name you want

import os
print(os.getcwd())

# Extract the zip file
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print(f"Extracted '{zip_path}' to '{extract_to}/'")

/sfs/gpfs/tardis/home/mwm6nq/DS4002/DS_4002_DataX_3/SCRIPTS
Extracted 'american-sign-language-letters-subdir.zip' to 'data/'


In [5]:
# Original dataset folder
SOURCE_DIR = "data/subdir"
# New output directories - train and test
TRAIN_VAL_DIR = "data/train_val"
TEST_DIR = "data/test"
# Test split ratio
TEST_SPLIT = 0.15  # 15% of images per class go to test

# Create target dirs if they don't exist
os.makedirs(TRAIN_VAL_DIR, exist_ok=True)
os.makedirs(TEST_DIR, exist_ok=True)

# Seed for reproducibility, a random split of data
random.seed(42)

# Loop through each class folder (A, B, C, ...)
for class_name in os.listdir(SOURCE_DIR):
    class_path = os.path.join(SOURCE_DIR, class_name)
    if not os.path.isdir(class_path):
        continue

    # List image files in the class folder
    image_files = [f for f in os.listdir(class_path) if f.lower().endswith((".jpg"))]
    #randomly shuffle the images in each folder
    random.shuffle(image_files)

    # Calculate how many to move to test using the 15% for test split and the count of how many images in each folder
    test_count = int(len(image_files) * TEST_SPLIT)
    print(test_count)

    # Define class folders in new dirs
    train_val_class_dir = os.path.join(TRAIN_VAL_DIR, class_name)
    test_class_dir = os.path.join(TEST_DIR, class_name)
    os.makedirs(train_val_class_dir, exist_ok=True)
    os.makedirs(test_class_dir, exist_ok=True)

    # Copies files to either test and train_val folder
    for i, file_name in enumerate(image_files):
        src_path = os.path.join(class_path, file_name)
        if i < test_count:
            dst_path = os.path.join(test_class_dir, file_name)
        else:
            dst_path = os.path.join(train_val_class_dir, file_name)
        shutil.copy2(src_path, dst_path)

print(test_count)


8
10
8
9
10
10
8
10
9
9
11
10
9
10
9
9
8
9
13
7
7
12
11
11
9
11
11


In [6]:
from tensorflow.keras.utils import image_dataset_from_directory
import numpy as np

# Load dataset
train_val_ds = image_dataset_from_directory("data/train_val", image_size=(64, 64), batch_size=32, color_mode="grayscale")

# Unbatch to get all individual labels
all_labels = []
for _, labels in train_val_ds.unbatch():
    all_labels.append(labels.numpy())

all_labels = np.array(all_labels)

Found 1481 files belonging to 26 classes.


2025-04-22 00:59:22.795289: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


In [7]:
from sklearn.utils.class_weight import compute_class_weight
#Some letters might have too few images
#Use class_weight to oversample smaller classes

class_names = train_val_ds.class_names  # list like ['A', 'B', ..., 'Z', 'space']

class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(all_labels),
    y=all_labels
)

class_weight_dict = dict(enumerate(class_weights))

In [8]:
#image information
data_dir = "data/subdir" #root directory
img_height, img_width = 64, 64 #resize images to 64x64
batch_size = 32 #number of images processed together in a single training step
#changing

In [5]:
nvidia-smi

NameError: name 'nvidia' is not defined

In [9]:

# Train/validation split from train_val folder
train_val_dir = "data/train_val"
test_dir = "data/test"

# Training set with using 80% of train_val data 
raw_train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_val_dir,
    validation_split=0.2,
    subset="training",
    seed=42,
    image_size=(img_height, img_width), #resizing the images 
    batch_size=batch_size,
    label_mode="int",
    color_mode="grayscale"
)

# Validation set with using 20% of train_val data 
raw_val_ds = tf.keras.preprocessing.image_dataset_from_directory(
    train_val_dir,
    validation_split=0.2,
    subset="validation",
    seed=42,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode="int",
    color_mode="grayscale"
)

# Test set — loaded from separate folder
test_ds = tf.keras.preprocessing.image_dataset_from_directory(
    test_dir,
    image_size=(img_height, img_width),
    batch_size=batch_size,
    label_mode="int",
    color_mode="grayscale"
)

Found 1481 files belonging to 26 classes.
Using 1185 files for training.
Found 1481 files belonging to 26 classes.
Using 296 files for validation.
Found 247 files belonging to 26 classes.


In [10]:
# Create a preprocessing function: normalize scales image of pixels
def preprocess(image, label):
    image = tf.cast(image, tf.float32) / 255.0
    return image, label


In [6]:
tf.config.list_physical_devices('GPU')

2025-04-22 01:15:11.280758: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


[]

In [7]:
# There is currently no official GPU support for MacOS.
python3 -m pip install tensorflow
# Verify the installation:
python3 -c "import tensorflow as tf; print(tf.reduce_sum(tf.random.normal([1000, 1000])))"

SyntaxError: invalid syntax (1493050825.py, line 2)

In [13]:
srun --partition=gpu --gres=gpu:1 --mem=16G --time=2:00:00 --pty bash

SyntaxError: invalid decimal literal (2086100770.py, line 1)

In [88]:
# Apply preprocessing function to train, validation, and test ds
train_ds = raw_train_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = raw_val_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocess, num_parallel_calls=tf.data.AUTOTUNE)


In [89]:
# tools to create the model to run faster
# stores images in memory after the first epoch,  randomizes training data order, overlaps data loading with model training for better performance

AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.cache().shuffle(1000).prefetch(buffer_size=AUTOTUNE) 
val_ds = val_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [90]:
#uses float16 for fast training and using less memory
#using a Mixed Precision Training
from tensorflow.keras import mixed_precision
mixed_precision.set_global_policy("mixed_float16")

In [91]:
# Define data augmentation as a layer
# every training batch is randomly altered to improve generalization.
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
], name="data_augmentation")

In [92]:
from tensorflow.keras import Sequential
from tensorflow.keras.layers import (Input, Rescaling, Resizing, Lambda, Concatenate, SeparableConv2D, BatchNormalization, 
                                     GlobalAveragePooling2D, Dropout, Dense)
import tensorflow as tf

using_pretrained = False  # Set to True if you switch to MobileNetV2 or similar

#builds a lightweight convolutional neural network (CNN), a custom model

#defines the shape of the model, a 64x64 grayscale 
# the model then takes each ASL sign image and extracts features using efficient convolutional layers
#Reduces dimensions of image using global pooling
#Learns final patterns with Dense layers
#outputs a probability for each letter (A–Z + space)
# Full model with data augmentation
model_layers = [
    Input(shape=(64, 64, 1)),
    Rescaling(1./255),
    Resizing(64, 64)
]

# Optional: Convert grayscale to RGB by duplicating channels
#if using_pretrained:
  #  model_layers.append(Concatenate(axis=-1)([
     #   Lambda(lambda x: x),
     #   Lambda(lambda x: x),
     #   Lambda(lambda x: x)
  #  ]))

# Add data augmentation
model_layers.append(data_augmentation)

# Core model layers
model_layers += [
    SeparableConv2D(32, 3, activation='relu'),
    BatchNormalization(),
    SeparableConv2D(128, 3, activation='relu'),
    BatchNormalization(),
    SeparableConv2D(128, 3, activation='relu'),
    BatchNormalization(),
    GlobalAveragePooling2D(),
    Dropout(0.3),
    Dense(128, activation='relu'),
    Dropout(0.3),
    Dense(64, activation='relu'),
    Dense(27, activation='softmax', dtype='float32')
]

# Build model
model = Sequential(model_layers)


In [93]:
#compile the model with Adam Optimizer with Learning Rate Schedule and a Loss Function for Integer Labels

#decreasing the learning rate during training
lr_schedule = tf.keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate=1e-4,
    decay_steps=1000,
    decay_rate=0.9)

model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=lr_schedule),
    loss='sparse_categorical_crossentropy', #Appropriate Loss Function for Integer Labels
    metrics=['accuracy']
    
)
# ensure numerical stability in loss scaling:

optimizer = tf.keras.optimizers.Adam(learning_rate=lr_schedule)
optimizer = mixed_precision.LossScaleOptimizer(optimizer)


In [94]:
#summary of the model
model.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_4 (Rescaling)     (None, 64, 64, 1)         0         
                                                                 
 resizing_4 (Resizing)       (None, 64, 64, 1)         0         
                                                                 
 data_augmentation (Sequent  (None, 64, 64, 1)         0         
 ial)                                                            
                                                                 
 separable_conv2d_15 (Separ  (None, 62, 62, 32)        73        
 ableConv2D)                                                     
                                                                 
 batch_normalization_15 (Ba  (None, 62, 62, 32)        128       
 tchNormalization)                                               
                                                      

In [95]:
#train the model

from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

#Stops training if validation loss doesn't improve for 30 epochs
callbacks = [
    EarlyStopping(patience=30, restore_best_weights=True),
    ReduceLROnPlateau(factor=0.5, patience=30, min_lr=1e-6)
]
#change number of epochs to imporve accuary of model
history = model.fit(
    train_val_ds,
    validation_data=val_ds,
    epochs=400,
    callbacks=callbacks
)


Epoch 1/400
47/47 [==============================] - 153s 3s/step - loss: 3.3488 - accuracy: 0.0419 - val_loss: 3.2953 - val_accuracy: 0.0541 - lr: 9.9517e-05
Epoch 2/400
47/47 [==============================] - 150s 3s/step - loss: 3.3023 - accuracy: 0.0520 - val_loss: 3.2946 - val_accuracy: 0.0439 - lr: 9.9025e-05
Epoch 3/400
47/47 [==============================] - 150s 3s/step - loss: 3.2920 - accuracy: 0.0540 - val_loss: 3.2938 - val_accuracy: 0.0439 - lr: 9.8536e-05
Epoch 4/400
47/47 [==============================] - 150s 3s/step - loss: 3.2681 - accuracy: 0.0655 - val_loss: 3.2929 - val_accuracy: 0.0439 - lr: 9.8049e-05
Epoch 5/400
47/47 [==============================] - 151s 3s/step - loss: 3.2650 - accuracy: 0.0641 - val_loss: 3.2917 - val_accuracy: 0.0439 - lr: 9.7565e-05
Epoch 6/400
47/47 [==============================] - 151s 3s/step - loss: 3.2567 - accuracy: 0.0736 - val_loss: 3.2908 - val_accuracy: 0.0439 - lr: 9.7083e-05
Epoch 7/400
47/47 [===========================

KeyboardInterrupt: 

In [75]:
#evaluate on test set
test_loss, test_acc = model.evaluate(test_ds)
print(f"✅ Final Test Accuracy: {test_acc:.4f}")


RuntimeError: You must compile your model before training/testing. Use `model.compile(optimizer, loss)`.

In [76]:
import matplotlib.pyplot as plt
# clean line plot to visualize how your model’s loss changes over time on both the training and validation sets
#ideal graph has training loss and validation loss both decrease over time, and when the curves flatten the model learning has stablized

# Extract loss values
train_loss = history.history['loss']
val_loss = history.history['val_loss']

# Plotting
plt.plot(train_loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training vs Validation Loss')
plt.legend()
plt.show()

NameError: name 'history' is not defined